# Docs
- [the Consumer interface](https://docs.oracle.com/javase/8/docs/api/java/util/function/Consumer.html)
- [the Function interface](https://docs.oracle.com/javase/8/docs/api/java/util/function/Function.html)
- [the Comparator interface](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Comparator.html)
- [Stream.map](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html#map-java.util.function.Function-)
- [Stream.collect](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html#collect-java.util.function.Supplier-java.util.function.BiConsumer-java.util.function.BiConsumer-)
- [IntStream.forEach](https://docs.oracle.com/javase/8/docs/api/java/util/stream/IntStream.html#forEach-java.util.function.IntConsumer-)
- [IntStream.peek](https://docs.oracle.com/javase/8/docs/api/java/util/stream/IntStream.html#peek-java.util.function.IntConsumer-)
- [List.sort](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/List.html#sort(java.util.Comparator))
- [ArrayList.forEach](https://docs.oracle.com/javase/8/docs/api/java/util/ArrayList.html#forEach-java.util.function.Consumer-)

## Tutorials
- [Lambda Expressions](https://docs.oracle.com/javase/tutorial/java/javaOO/lambdaexpressions.html)

# Basics

## Lambdas as implementations of arbitrary single function ("functional") interfaces

In [41]:
interface Doable {
    void execute();
}

null

In [47]:
class MyDoable implements Doable {
    public void execute() {
        System.out.println("Hi!");
    }
};

Doable a = new MyDoable();

a.execute()

Hi!


null

In previous versions of Java, you could use an anonymous class implementation:

In [43]:
Doable a = new Doable() {
    public void execute() {
        System.out.println("Hi!");
    }
};

a.execute()

Hi!


null

After Java 8, you can use lambda expressions instead:

In [45]:
Doable a = () -> {
    System.out.println("Hi!");
};

a.execute()

Hi!


null

## Examples

In [30]:
var arange = IntStream.range(0, 10)

java.util.stream.IntPipeline$Head@489b1d29

In [31]:
var arangeList = arange.boxed().collect(Collectors.toList()); // converting the IntStream to a normal (boxed) stream, then converting it to a List
arangeList;

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [36]:
Consumer fn = (x) -> System.out.println(x);
arangeList.forEach(fn); 

0
1
2
3
4
5
6
7
8
9


null

Type inference will be done automatically if the lambda expression is used directly in a method call (as the compiler knows the type signature needed):

In [37]:
arangeList.forEach((x) -> System.out.println(x)); 

0
1
2
3
4
5
6
7
8
9


null

But the compiler is not smart enough for this kind of type inference:

In [38]:
var fn = (x) -> System.out.println(x);
arange.forEach(fn);

var fn = (x) -> System.out.println(x);: var fn = (x) -> System.out.println(x);

It's possible to use the general type `Function`, but then it might not be compatible with the functional interface needed by the caller:

In [43]:
Function fn = (x) -> { 
    System.out.println(x);
    return null;
    };

arangeList.forEach(fn);

: 

Now let's see a pure function that is used for computation instead of side-effects.

We first have to convert our `List` to a `Stream`, then convert it back into a `List`. This is because Java's `List` does not have a `map` method.

In [59]:
arangeList.stream().map((x) -> {
   return x*10; 
}).collect(Collectors.toList());

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

We can use the generic type `Function` for these; but we will still lose some type information. This causes us to need to cast `x` to `int` explicitly.

In [65]:
Function fn = (x) -> {
    var x_int = (int)x;
    return x_int*10;
};

arangeList.stream().map(fn).collect(Collectors.toList());

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

Let's define a helper function to run `map` on `List` more concisely:

In [68]:
List listMap(List lst, Function fn) {
    return (List)(lst.stream().map(fn).collect(Collectors.toList()));
}

null

In [69]:
Function fn = (x) -> {
    var x_int = (int)x;
    return x_int*10;
};

listMap(arangeList, fn);

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

# Stream functions

In [106]:
var arange = IntStream.range(0, 10); // We need to recreate this `stream` every time we use it, as it will be "consumed."

arange
  .filter((x) -> (x % 2 == 0))
  .toArray();

[0, 2, 4, 6, 8]

In [110]:
var arange = IntStream.range(0, 10); 
arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .toArray();

[2, 4, 6, 8]

In [113]:
var arange = IntStream.range(0, 10); 
arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .map((x) -> x*3)
  .toArray();

[6, 12, 18, 24]

In [121]:
var arange = IntStream.range(0, 10); 

arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .map((x) -> x*3)
  .reduce(0, (a, b) -> a+b); // https://docs.oracle.com/javase/8/docs/api/java/util/stream/IntStream.html#reduce-int-java.util.function.IntBinaryOperator-

60

We could also use the function reference `Integer::sum` for the last reduction:

In [123]:
Integer.sum(8,11); // an example

19

In [122]:
var arange = IntStream.range(0, 10); 

arange
  .filter((x) -> (x % 2 == 0))
  .filter((x) -> (x > 0))
  .map((x) -> x*3)
  .reduce(0, Integer::sum);

60

# Function composition

## Composing functions returning Void (i.e., running functions for their side-effects)

In [48]:
Consumer consumer_sequential_combination(List<Consumer> consumers) {
    return (x) -> {
        consumers.forEach((consumer) -> consumer.accept(x)); 
        // We looked at the docs to know that this interface uses the name `accept` for its "main" method.
    };
}

null

In [54]:
Consumer fn_print = (x) -> System.out.println(x);
Consumer fn_double_print = (x) -> {
    int x_int = (int)x;
    System.out.println(x_int*2);
};
Consumer fn_print_sep = (x) -> System.out.println("#####");

Consumer fn_all = consumer_sequential_combination(Arrays.asList(fn_print, fn_double_print, fn_print_sep));

arangeList.forEach(fn_all); 

0
0
#####
1
2
#####
2
4
#####
3
6
#####
4
8
#####
5
10
#####
6
12
#####
7
14
#####
8
16
#####
9
18
#####


null

Exercise: Create `consumer_parallel_combination` that runs its consumers in parallel.

## Composing pure functions returning results

In [80]:
Function fn_compose(List<Function> functions) {
    return (x) -> {
        Object res = x;
        for (Function fn : functions) {
            res = fn.apply(res);
            // https://docs.oracle.com/javase/8/docs/api/java/util/function/Function.html
        }
        return res;
    };
}

null

In [90]:
Function fn_double = (x) -> {
    var x_int = (int)x;
    return x_int * 10;
};

Function fn_inc = (x) -> {
    var x_int = (int)x;
    return x_int + 1;
};

Function fn_all = fn_compose(Arrays.asList(fn_double, fn_inc, fn_inc, fn_inc));

listMap(arangeList, fn_all);

[3, 13, 23, 33, 43, 53, 63, 73, 83, 93]

# Using lambdas for sorting

In [100]:
arangeList.sort((a, b) -> {
   if (a == 3) {
       return -1; // '3' is smaller than everything else.
   } else if (b == 3) {
       return 1; // everything is bigger than '3'.
   } else if (a < b) {
       return -1;
   } else if (a == b) {
       return 0;
   } else {
       return 1;
   }
});

arangeList;

[3, 0, 1, 2, 4, 5, 6, 7, 8, 9]

We can use the following helper function [Comparator.comparingInt](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Comparator.html#comparingInt(java.util.function.ToIntFunction)) which makes the code cleaner.

`static Comparator`   `comparingInt​(ToIntFunction<? super T> keyExtractor)`:   Accepts a function that extracts an `int` sort key from a type `T`, and returns a `Comparator` that compares by that sort key.

In [103]:
arangeList.sort(Comparator.comparingInt((x) -> {
    if (x == 3) {
        return Integer.MIN_VALUE;
    }
    
    return x;
}));

arangeList;

[3, 0, 1, 2, 4, 5, 6, 7, 8, 9]

# Closures

Definition: A "closure" is something (a function or object or other thing that can be run in some way like having methods) that captures ("closes over") a local variable from its enclosing scope, and which can use that variable in its code, even when the function or object's method is run at a later time, including when the enclosing scope no longer exists.

Java does not support closures that hold references to the captured variables.

See:
- https://stackoverflow.com/questions/3805474/what-is-a-closure-does-java-have-closures
- https://stackoverflow.com/questions/17204279/does-java-8-support-closures

In [37]:
void f1() {
    int number = 5;

    Function callback = (x) -> {
        return (number + (int)x);
    };

    Object result = callback.apply(10);

    System.out.println(result);
    System.out.println(number);
}

f1();

15
5


null

In [18]:
void f1() {
    int number = 5;

    Function callback = (x) -> {
        return (number = number + (int)x);
    };

    Object result = callback.apply(10);

    System.out.println(result);
    System.out.println(number);
}

f1();

void f1() {: void f1() {

### @skipMe The REPL here is funny

In [10]:
int number = 5;

Function callback = (x) -> {
    return (number = number + (int)x);
};

Object result = callback.apply(10);

System.out.println(result);
System.out.println(number);

15
15


null

In [31]:
int f1() {
    final int a = 6;
    a = 2;
    return a;
}

f1();

int f1() {: int f1() {

In [34]:
int f1() {
    final static int a = 6;
    a = 2;
    return a;
}

f1();

int f1() {: int f1() {

In [33]:
final int a = 6;
a = 2;
a;

2

# Immutable collections

## Java's own

The REPL here is buggy, so I have attached the correct output (which you can test via `jshell`) manually.

In [54]:
List f1() {
    List<String> list = new ArrayList<>(Arrays.asList("one", "two", "three"));
    List<String> unmodifiableList = Collections.unmodifiableList(list);
    unmodifiableList.add("four");
    return unmodifiableList;
}

f1();

null

```
|  Exception java.lang.UnsupportedOperationException
|        at Collections$UnmodifiableCollection.add (Collections.java:1062)
|        at f1 (#6:4)
|        at (#7:1)
```

### Updating the immutable collections (into new immutable collections)

In [55]:
List<String> list = new ArrayList<>(Arrays.asList("one", "two", "three"));
List<String> unmodifiableList = Collections.unmodifiableList(list);

Stream.concat(unmodifiableList.stream(),
              Stream.of("four"))
      .collect(Collectors.toUnmodifiableList());

[one, two, three, four]

# When is functional programming useful?

- concurrency and parallelism
  + immutable data needs no synchronization
  + the map-reduce pattern
- immutable data are stable regarding equals and hashCode and thus are reliable hash keys
- pure functions are easier to understand and provide less leaky abstractions
- lazy computations are more convenient in a functional context
- progrmming flexibility and metaprogramming
  + functions are easier to compose than objects
  + functions and objects aren't that different
    - a method call can be otherwise seen as a function call with the first argument being the object (like Python's `self`)
    - the essense of an object is
      + behavior (methods)
      + polymorphism, inheritance, interfaces
      + data
    - the equivalent of the above are roughly
      + the function itself
      + multiple dispatch (which needs a type system that is not much different from normal OOP practices)
      + data classes, structs, basic collections (dictionaries, lists, etc.)
        - the data can be stored in a closure
    - concrete inheritance can be a problem in functional contexts that disavow OOP
      + mix-ins
      + traits
    - modern languages are adding functional elements to classic OOP
      + perhaps this is because having zero overhead functional conveniences are harder on the compiler
- declarative programming
- tracing compilers
  + accelerated deep learning via JAX
- mathematical operations
  + getting the derivative of a function
- some languages have no object system, e.g., bash
- functional programming is generally easier in dynamic languages

# Popular functional libraries in Java

-   [Cyclops](https://github.com/aol/cyclops) - Monad and stream utilities, comprehensions, pattern matching, functional extensions for all JDK collections, future streams, trampolines and much more.
-   [derive4j](https://github.com/derive4j/derive4j) - Java 8 annotation processor and framework for deriving algebraic data types constructors, pattern-matching and morphisms. (GPL-3.0-only)
-   [Fugue](https://bitbucket.org/atlassian/fugue) - Functional extensions to Guava.
-   [Functional Java](http://www.functionaljava.org/) - Implements numerous basic and advanced programming abstractions that assist composition-oriented development.
-   [jOOλ](https://github.com/jOOQ/jOOL) - Extension to Java 8 that aims to fix gaps in lambda by providing numerous missing types and a rich set of sequential Stream API additions.
-   [protonpack](https://github.com/poetix/protonpack) - Collection of stream utilities.
-   [StreamEx](https://github.com/amaembo/streamex) - Enhances Java 8 Streams.
-   [Vavr](https://www.vavr.io/) - Functional component library that provides persistent data types and functional control structures.
  + [userguide](https://docs.vavr.io/)


## @skipMe Vavr

In [58]:
%%classpath add mvn
<dependencies>
    <dependency>
        <groupId>io.vavr</groupId>
        <artifactId>vavr</artifactId>
        <version>0.10.4</version>
    </dependency>
</dependencies>

In [65]:
%import io.vavr.collection

// io.vavr.collection.List<Integer> list1 = io.vavr.collection.List.of(1, 2, 3);
// io.vavr.collection.List<Integer> list2 = list1.tail().prepend(0);

Could not import io.vavr.collection


Index 0 out of bounds for length 0: Index 0 out of bounds for length 0

# Scala

In the Scala implementation, a cons-cell of type T is an object of type ::[T]. It has two val-fields, namely head and tail. The tail of a cons-cell of type T contains a reference to the unique object Nil, or to another cons-cell of type T.

The Scala kernel does not currently work, but we can still see the idea of a cons cell:

In [1]:
%%scala

val mainList = List(3, 2, 1)
val with4 =    4 :: mainList  // re-uses mainList, costs one :: instance
val with42 =   42 :: mainList // also re-uses mainList, cost one :: instance
val shorter =  mainList.tail  // costs nothing as it uses the same 2::1::Nil instances as mainList

py4j.Py4JException:  An exception was raised by the Python Proxy. Return Message

# Tail call optimization in Java

- https://github.com/Judekeyser/tail_rec : a proof-of-concept TCO implementation in Java
  + https://scribe.rip/javarevisited/tail-recursion-in-java-abc24f56b56b

# Tmp